# Set the environment

In [1]:
# set the envoiroment
import cobra
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis import flux_variability_analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob 
import escher
from escher import Builder
from utils import show_map
from utils.check_precursor_problem import check_precursor_problem

# Load the model

In [2]:
model = cobra.io.load_json_model("../models/updated_model.json")# Import the model.
model.solver = 'glpk'
model.reactions.T_flux_oxygen.bounds = (0,0) # block the import of oxygen
model.reactions.NGAM.bounds = (0,1000)
map_loc = "../data/small_pathway.json"

# The small model 

In [3]:
M= model.copy()

## add the reaction for the maintainance of NADPH
reaction = Reaction('NADPH_recycle')
reaction.name = 'NADPH_recycle'
reaction.lower_bound = -100.  # This is the default
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({'NADPH': -1.0,
                         'NADP+': 1.0,
                         "H+":1.0})

reaction.reaction 
## add the reaction for the maintainance of NADH
reaction = Reaction('NADH_recycle')
reaction.name = 'NADH_recycle'
reaction.lower_bound = -100.  # This is the default
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({'NADH': -1.0,
                         'NAD+': 1.0,
                         "H+":1.0})
reaction.reaction
reaction = Reaction('T_flux_pyruvate')
reaction.name = 'T_flux_pyruvate'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({'pyruvate': -1.0
                         })

reaction = Reaction('ATP_source_reaction')
reaction.name = 'ATP_source_reaction'
reaction.lower_bound = -0.  # This is the default
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"ADP": -1.0,
                          "H+": -1.0,
                          "phosphate":-1.0,
                          "H2O": 1.0,
                         "ATP": 1.0})
total_reactions_id = [r.id for r in M.reactions]
small_pathway_reactions = ['T_flux_beta__D__glucose','T_ABC__transporter_beta__D__glucose',
'carb_entner_1.1.1.359_GLUCOSE__1__DEHYDROGENASE__NAD+__RXN','carb_entner_1.1.1.47_GLUCOSE__1__DEHYDROGENASE__NAD+__RXN',
'carb_entner_1.1.1.360_GLUCOSE__1__DEHYDROGENASE__NADP+__RXN','carb_entner_3.1.1.17_GLUCONOLACT__RXN',
'carb_entner_4.2.1.140_GLUCONATE__DEHYDRATASE__RXN','carb_degra__glucose_4.2.1.9_GLUCONATE__DEHYDRATASE__RXN','carb_entner_2.7.1.178_DEOXYGLUCONOKIN__RXN',
'carb_entner_4.1.2.55_KDPGALDOL__RXN',"carb_glucneo_1.2.1.13_GAPDHSYNEC__RXN_NAD","carb_glucneo_1.2.1.13_GAPDHSYNEC__RXN_NADP","carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN",
 "carb_enter_glucneo_1.2.1.9_1.2.1.9__RXN",'T_flux_H+','T_trans_H+','T_flux_H2O','T_trans_H2O','T_flux_phosphate','T_trans_phosphate',
'carb_enter_glucneo_5.4.2.1_3PGAREARR__RXN','carb_enter_glucneo_4.2.1.11_2PGADEHYDRAT__RXN',
'carb_enter_2.7.1.40_PEPDEPHOS__RXN','NADPH_recycle','NADH_recycle','T_flux_pyruvate','ATP_source_reaction']

for r in total_reactions_id:
    if r not in small_pathway_reactions:
        M.reactions.get_by_id(r).upper_bound = 0
        M.reactions.get_by_id(r).lower_bound = 0
    

M.objective = M.reactions.get_by_id('T_flux_pyruvate')
sol = M.optimize()
b = show_map(sol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

# Import the mRNA data and Vmax data

In [4]:
df_gene_reaction_rule =pd.read_csv('../data/reaction_id_RAS 4.1.csv')
# set the reaction id column as the index in order to map it to the model
df_gene_reaction_rule = df_gene_reaction_rule.fillna(1000)
df_gene_reaction_rule = df_gene_reaction_rule.set_index(["reaction_id"],)

# Write a function for mapping transcriptomics to model reaction bounds

In [5]:
def map_the_RNA_to_model (a, b,c):# Here a is the number used to divid the transcriptome data, b is the column for the transcriptome data, -c is the lower bound for glucose exchange reaction 
    model1 = M.copy()
    model1.reactions.get_by_id('T_flux_beta__D__glucose').bounds = (-c,0)

    for rxn in small_pathway_reactions:
        if rxn in df_gene_reaction_rule.index:

            the_bound = df_gene_reaction_rule[b].loc[rxn]/a

            if model1.reactions.get_by_id(rxn).lower_bound < 0 and model1.reactions.get_by_id(rxn).upper_bound > 0: # this finds reversible rxns

                model1.reactions.get_by_id(rxn).bounds = (-1*the_bound, the_bound) # we set the bound(s) with max abs(FVA) 

            elif model1.reactions.get_by_id(rxn).upper_bound > 0: # irreversible, flows in forward direction

                model1.reactions.get_by_id(rxn).bounds = (0, the_bound)  # the lower bound will stay zero

            elif int(model1.reactions.get_by_id(rxn).lower_bound) < 0: # irreversible, flows in backward direction

                model1.reactions.get_by_id(rxn).bounds = (-1*the_bound, 0) # the upper bound will stay zero
    model1.objective  = model1.reactions.get_by_id("T_flux_pyruvate")
    sol = model1.optimize()
    l = sol.fluxes[abs(sol.fluxes)>1e-6].index
    for rxn in l:
        print (rxn,model1.reactions.get_by_id(rxn).bounds,sol.fluxes.loc[rxn])
    b = show_map(sol,map_loc)
    return(b)

# Map the transcriptomics at different temperatures 70 and 80 ℃

## 70 ℃, different a factor 

### 70 ℃, a factor was  fixed at 100000

In [6]:
map_the_RNA_to_model (100000,'mRNA (70)',1)

T_flux_H+ (-1000, 1000) 2.981320000000001
T_trans_H+ (-1000, 1000) 2.9813200000000006
T_flux_beta__D__glucose (-1, 0) -0.74533
T_ABC__transporter_beta__D__glucose (0, 1.844567) 0.7453300000000003
carb_entner_1.1.1.360_GLUCOSE__1__DEHYDROGENASE__NADP+__RXN (0, 1.31251) 0.7453300000000003
carb_entner_3.1.1.17_GLUCONOLACT__RXN (0, 10.0) 0.7453300000000004
carb_entner_4.2.1.140_GLUCONATE__DEHYDRATASE__RXN (0, 8.365715) 0.7453300000000004
carb_entner_2.7.1.178_DEOXYGLUCONOKIN__RXN (0, 2.912728) 0.7453300000000003
carb_entner_4.1.2.55_KDPGALDOL__RXN (0, 5.973186999999999) 0.7453300000000002
carb_enter_glucneo_5.4.2.1_3PGAREARR__RXN (-0.74533, 0.74533) 0.74533
carb_enter_glucneo_4.2.1.11_2PGADEHYDRAT__RXN (-4.107753, 4.107753) 0.74533
carb_enter_2.7.1.40_PEPDEPHOS__RXN (0, 1.92121) 0.74533
carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN (-5.995728000000001, 5.995728000000001) -0.7453300000000002
carb_glucneo_1.2.1.13_GAPDHSYNEC__RXN_NAD (-6.530183, 6.530183) -0.7453300000000002
NADPH_recycle (-10

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### 70 ℃, a factor was  fixed at 10000

In [7]:
map_the_RNA_to_model (10000,'mRNA (70)',1)

T_flux_H+ (-1000, 1000) 4.000000000000002
T_trans_H+ (-1000, 1000) 4.000000000000001
T_flux_beta__D__glucose (-1, 0) -1.0
T_ABC__transporter_beta__D__glucose (0, 18.44567) 1.0000000000000002
carb_entner_1.1.1.360_GLUCOSE__1__DEHYDROGENASE__NADP+__RXN (0, 13.1251) 1.0000000000000002
carb_entner_3.1.1.17_GLUCONOLACT__RXN (0, 100.0) 1.0000000000000002
carb_entner_4.2.1.140_GLUCONATE__DEHYDRATASE__RXN (0, 83.65715) 1.0000000000000002
carb_entner_2.7.1.178_DEOXYGLUCONOKIN__RXN (0, 29.12728) 1.0000000000000002
carb_entner_4.1.2.55_KDPGALDOL__RXN (0, 59.731869999999994) 1.0
carb_enter_glucneo_5.4.2.1_3PGAREARR__RXN (-7.4533, 7.4533) 1.0000000000000002
carb_enter_glucneo_4.2.1.11_2PGADEHYDRAT__RXN (-41.077529999999996, 41.077529999999996) 1.0000000000000002
carb_enter_2.7.1.40_PEPDEPHOS__RXN (0, 19.2121) 1.0000000000000002
carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN (-59.957280000000004, 59.957280000000004) -1.0000000000000004
carb_glucneo_1.2.1.13_GAPDHSYNEC__RXN_NAD (-65.30183000000001, 65.3

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### 70 ℃, a factor was  fixed at 1000000

In [8]:
map_the_RNA_to_model (1000000,'mRNA (70)',1)

T_flux_H+ (-1000, 1000) 0.298132
T_trans_H+ (-1000, 1000) 0.298132
T_flux_beta__D__glucose (-1, 0) -0.07453299999999996
T_ABC__transporter_beta__D__glucose (0, 0.1844567) 0.07453299999999996
carb_entner_1.1.1.360_GLUCOSE__1__DEHYDROGENASE__NADP+__RXN (0, 0.131251) 0.07453299999999999
carb_entner_3.1.1.17_GLUCONOLACT__RXN (0, 1.0) 0.074533
carb_entner_4.2.1.140_GLUCONATE__DEHYDRATASE__RXN (0, 0.8365715) 0.074533
carb_entner_2.7.1.178_DEOXYGLUCONOKIN__RXN (0, 0.2912728) 0.07453299999999999
carb_entner_4.1.2.55_KDPGALDOL__RXN (0, 0.5973187) 0.07453299999999997
carb_enter_glucneo_5.4.2.1_3PGAREARR__RXN (-0.074533, 0.074533) 0.074533
carb_enter_glucneo_4.2.1.11_2PGADEHYDRAT__RXN (-0.4107753, 0.4107753) 0.07453299999999996
carb_enter_2.7.1.40_PEPDEPHOS__RXN (0, 0.192121) 0.07453299999999996
carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN (-0.5995728, 0.5995728) -0.07453299999999996
carb_glucneo_1.2.1.13_GAPDHSYNEC__RXN_NAD (-0.6530183, 0.6530183) -0.07453299999999997
NADPH_recycle (-100.0, 1000.

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

## 80 ℃, different a factor 

### 80 ℃, a factor was  fixed at 100000

In [9]:
map_the_RNA_to_model (100000,'mRNA (80)',1)

T_flux_H+ (-1000, 1000) 4.000000000000002
T_trans_H+ (-1000, 1000) 4.000000000000001
T_flux_beta__D__glucose (-1, 0) -1.0
T_ABC__transporter_beta__D__glucose (0, 1.01448) 1.0000000000000002
carb_entner_1.1.1.360_GLUCOSE__1__DEHYDROGENASE__NADP+__RXN (0, 1.16348) 1.0000000000000002
carb_entner_3.1.1.17_GLUCONOLACT__RXN (0, 10.0) 1.0000000000000002
carb_entner_4.2.1.140_GLUCONATE__DEHYDRATASE__RXN (0, 3.56251) 1.0000000000000002
carb_entner_2.7.1.178_DEOXYGLUCONOKIN__RXN (0, 2.5335579999999998) 1.0000000000000002
carb_entner_4.1.2.55_KDPGALDOL__RXN (0, 3.46968) 1.0
carb_enter_glucneo_5.4.2.1_3PGAREARR__RXN (-1.004765, 1.004765) 1.0000000000000002
carb_enter_glucneo_4.2.1.11_2PGADEHYDRAT__RXN (-4.253073, 4.253073) 1.0000000000000002
carb_enter_2.7.1.40_PEPDEPHOS__RXN (0, 3.07618) 1.0000000000000002
carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN (-4.929608, 4.929608) -1.0000000000000004
carb_glucneo_1.2.1.13_GAPDHSYNEC__RXN_NAD (-6.758985, 6.758985) -1.0000000000000002
NADPH_recycle (-100.0, 

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### 80 ℃, a factor was  fixed at 10000

In [10]:
map_the_RNA_to_model (10000,'mRNA (80)',1)

T_flux_H+ (-1000, 1000) 4.000000000000002
T_trans_H+ (-1000, 1000) 4.000000000000001
T_flux_beta__D__glucose (-1, 0) -1.0
T_ABC__transporter_beta__D__glucose (0, 10.1448) 1.0000000000000002
carb_entner_1.1.1.360_GLUCOSE__1__DEHYDROGENASE__NADP+__RXN (0, 11.6348) 1.0000000000000002
carb_entner_3.1.1.17_GLUCONOLACT__RXN (0, 100.0) 1.0000000000000002
carb_entner_4.2.1.140_GLUCONATE__DEHYDRATASE__RXN (0, 35.6251) 1.0000000000000002
carb_entner_2.7.1.178_DEOXYGLUCONOKIN__RXN (0, 25.33558) 1.0000000000000002
carb_entner_4.1.2.55_KDPGALDOL__RXN (0, 34.6968) 1.0
carb_enter_glucneo_5.4.2.1_3PGAREARR__RXN (-10.04765, 10.04765) 1.0000000000000002
carb_enter_glucneo_4.2.1.11_2PGADEHYDRAT__RXN (-42.53073, 42.53073) 1.0000000000000002
carb_enter_2.7.1.40_PEPDEPHOS__RXN (0, 30.7618) 1.0000000000000002
carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN (-49.296079999999996, 49.296079999999996) -1.0000000000000004
carb_glucneo_1.2.1.13_GAPDHSYNEC__RXN_NAD (-67.58985, 67.58985) -1.0000000000000002
NADPH_recycl

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### 80 ℃, a factor was  fixed at 1000000

In [11]:
map_the_RNA_to_model (1000000,'mRNA (80)',1)

T_flux_H+ (-1000, 1000) 0.4019059999999999
T_trans_H+ (-1000, 1000) 0.4019059999999999
T_flux_beta__D__glucose (-1, 0) -0.10047649999999995
T_ABC__transporter_beta__D__glucose (0, 0.101448) 0.10047649999999995
carb_entner_1.1.1.360_GLUCOSE__1__DEHYDROGENASE__NADP+__RXN (0, 0.116348) 0.10047649999999995
carb_entner_3.1.1.17_GLUCONOLACT__RXN (0, 1.0) 0.10047649999999997
carb_entner_4.2.1.140_GLUCONATE__DEHYDRATASE__RXN (0, 0.356251) 0.10047649999999998
carb_entner_2.7.1.178_DEOXYGLUCONOKIN__RXN (0, 0.25335579999999996) 0.10047649999999998
carb_entner_4.1.2.55_KDPGALDOL__RXN (0, 0.346968) 0.10047649999999997
carb_enter_glucneo_5.4.2.1_3PGAREARR__RXN (-0.1004765, 0.1004765) 0.1004765
carb_enter_glucneo_4.2.1.11_2PGADEHYDRAT__RXN (-0.4253073, 0.4253073) 0.10047649999999995
carb_enter_2.7.1.40_PEPDEPHOS__RXN (0, 0.307618) 0.10047649999999995
carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN (-0.4929608, 0.4929608) -0.10047649999999995
carb_glucneo_1.2.1.13_GAPDHSYNEC__RXN_NAD (-0.6758985, 0.675898

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

# Map the Vmax at different temperatures 70 and 80 ℃

## 80 ℃ , different a factor 

### 80 ℃, a factor was fixed at 100

In [12]:
map_the_RNA_to_model (100,'80 (80) enzyme activities',1)

T_flux_H+ (-1000, 1000) 3.0000000000000004
T_trans_H+ (-1000, 1000) 3.0000000000000004
T_flux_beta__D__glucose (-1, 0) -0.7499999999999999
T_ABC__transporter_beta__D__glucose (0, 10.0) 0.75
carb_entner_1.1.1.360_GLUCOSE__1__DEHYDROGENASE__NADP+__RXN (0, 2.5) 0.75
carb_entner_3.1.1.17_GLUCONOLACT__RXN (0, 10.0) 0.7500000000000001
carb_entner_4.2.1.140_GLUCONATE__DEHYDRATASE__RXN (0, 1.7) 0.7500000000000001
carb_entner_2.7.1.178_DEOXYGLUCONOKIN__RXN (0, 10.0) 0.7500000000000001
carb_entner_4.1.2.55_KDPGALDOL__RXN (0, 1.2) 0.75
carb_enter_glucneo_5.4.2.1_3PGAREARR__RXN (-0.75, 0.75) 0.75
carb_enter_glucneo_4.2.1.11_2PGADEHYDRAT__RXN (-10.0, 10.0) 0.75
carb_enter_2.7.1.40_PEPDEPHOS__RXN (0, 0.76) 0.7499999999999998
carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN (-1.1, 1.1) -0.7500000000000002
carb_glucneo_1.2.1.13_GAPDHSYNEC__RXN_NAD (-0.87, 0.87) -0.75
NADPH_recycle (-100.0, 1000.0) 0.7500000000000001
NADH_recycle (-100.0, 1000.0) 0.7499999999999999
T_flux_pyruvate (0.0, 1000.0) 1.4999999999

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### 80 ℃, a factor was fixed at 1000

In [13]:
map_the_RNA_to_model (1000,'80 (80) enzyme activities',1)

T_flux_H+ (-1000, 1000) 0.3
T_trans_H+ (-1000, 1000) 0.3
T_flux_beta__D__glucose (-1, 0) -0.07499999999999997
T_ABC__transporter_beta__D__glucose (0, 1.0) 0.075
carb_entner_1.1.1.360_GLUCOSE__1__DEHYDROGENASE__NADP+__RXN (0, 0.25) 0.07499999999999998
carb_entner_3.1.1.17_GLUCONOLACT__RXN (0, 1.0) 0.075
carb_entner_4.2.1.140_GLUCONATE__DEHYDRATASE__RXN (0, 0.17) 0.075
carb_entner_2.7.1.178_DEOXYGLUCONOKIN__RXN (0, 1.0) 0.075
carb_entner_4.1.2.55_KDPGALDOL__RXN (0, 0.12) 0.075
carb_enter_glucneo_5.4.2.1_3PGAREARR__RXN (-0.075, 0.075) 0.075
carb_enter_glucneo_4.2.1.11_2PGADEHYDRAT__RXN (-1.0, 1.0) 0.075
carb_enter_2.7.1.40_PEPDEPHOS__RXN (0, 0.076) 0.075
carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN (-0.11, 0.11) -0.07500000000000001
carb_glucneo_1.2.1.13_GAPDHSYNEC__RXN_NAD (-0.087, 0.087) -0.075
NADPH_recycle (-100.0, 1000.0) 0.075
NADH_recycle (-100.0, 1000.0) 0.07499999999999998
T_flux_pyruvate (0.0, 1000.0) 0.15


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### 80 ℃, a factor was fixed at 10

In [14]:
map_the_RNA_to_model (10,'80 (80) enzyme activities',1)

T_flux_H+ (-1000, 1000) 4.000000000000002
T_trans_H+ (-1000, 1000) 4.000000000000001
T_flux_beta__D__glucose (-1, 0) -1.0
T_ABC__transporter_beta__D__glucose (0, 100.0) 1.0000000000000002
carb_entner_1.1.1.360_GLUCOSE__1__DEHYDROGENASE__NADP+__RXN (0, 25.0) 1.0000000000000002
carb_entner_3.1.1.17_GLUCONOLACT__RXN (0, 100.0) 1.0000000000000002
carb_entner_4.2.1.140_GLUCONATE__DEHYDRATASE__RXN (0, 17.0) 1.0000000000000002
carb_entner_2.7.1.178_DEOXYGLUCONOKIN__RXN (0, 100.0) 1.0000000000000002
carb_entner_4.1.2.55_KDPGALDOL__RXN (0, 12.0) 1.0
carb_enter_glucneo_5.4.2.1_3PGAREARR__RXN (-7.5, 7.5) 1.0000000000000002
carb_enter_glucneo_4.2.1.11_2PGADEHYDRAT__RXN (-100.0, 100.0) 1.0000000000000002
carb_enter_2.7.1.40_PEPDEPHOS__RXN (0, 7.6) 1.0000000000000002
carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN (-11.0, 11.0) -1.0000000000000004
carb_glucneo_1.2.1.13_GAPDHSYNEC__RXN_NAD (-8.7, 8.7) -1.0000000000000002
NADPH_recycle (-100.0, 1000.0) 1.0
NADH_recycle (-100.0, 1000.0) 1.0000000000000002


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

## 80 ℃ , different glucose import

### 80 ℃, glucose import reaction lower bound was fixed at 1

In [15]:
map_the_RNA_to_model (100,'80 (80) enzyme activities',1)

T_flux_H+ (-1000, 1000) 3.0000000000000004
T_trans_H+ (-1000, 1000) 3.0000000000000004
T_flux_beta__D__glucose (-1, 0) -0.7499999999999999
T_ABC__transporter_beta__D__glucose (0, 10.0) 0.75
carb_entner_1.1.1.360_GLUCOSE__1__DEHYDROGENASE__NADP+__RXN (0, 2.5) 0.75
carb_entner_3.1.1.17_GLUCONOLACT__RXN (0, 10.0) 0.7500000000000001
carb_entner_4.2.1.140_GLUCONATE__DEHYDRATASE__RXN (0, 1.7) 0.7500000000000001
carb_entner_2.7.1.178_DEOXYGLUCONOKIN__RXN (0, 10.0) 0.7500000000000001
carb_entner_4.1.2.55_KDPGALDOL__RXN (0, 1.2) 0.75
carb_enter_glucneo_5.4.2.1_3PGAREARR__RXN (-0.75, 0.75) 0.75
carb_enter_glucneo_4.2.1.11_2PGADEHYDRAT__RXN (-10.0, 10.0) 0.75
carb_enter_2.7.1.40_PEPDEPHOS__RXN (0, 0.76) 0.7499999999999998
carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN (-1.1, 1.1) -0.7500000000000002
carb_glucneo_1.2.1.13_GAPDHSYNEC__RXN_NAD (-0.87, 0.87) -0.75
NADPH_recycle (-100.0, 1000.0) 0.7500000000000001
NADH_recycle (-100.0, 1000.0) 0.7499999999999999
T_flux_pyruvate (0.0, 1000.0) 1.4999999999

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### 80 ℃, glucose import reaction lower bound was fixed at 0.4

In [16]:
map_the_RNA_to_model (100,'80 (80) enzyme activities',0.4)

T_flux_H+ (-1000, 1000) 1.6000000000000003
T_trans_H+ (-1000, 1000) 1.6000000000000003
T_flux_beta__D__glucose (-0.4, 0) -0.4
T_ABC__transporter_beta__D__glucose (0, 10.0) 0.4
carb_entner_1.1.1.360_GLUCOSE__1__DEHYDROGENASE__NADP+__RXN (0, 2.5) 0.4
carb_entner_3.1.1.17_GLUCONOLACT__RXN (0, 10.0) 0.4000000000000001
carb_entner_4.2.1.140_GLUCONATE__DEHYDRATASE__RXN (0, 1.7) 0.4000000000000001
carb_entner_2.7.1.178_DEOXYGLUCONOKIN__RXN (0, 10.0) 0.4000000000000001
carb_entner_4.1.2.55_KDPGALDOL__RXN (0, 1.2) 0.4000000000000001
carb_enter_glucneo_5.4.2.1_3PGAREARR__RXN (-0.75, 0.75) 0.4
carb_enter_glucneo_4.2.1.11_2PGADEHYDRAT__RXN (-10.0, 10.0) 0.4
carb_enter_2.7.1.40_PEPDEPHOS__RXN (0, 0.76) 0.4
carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN (-1.1, 1.1) -0.4
carb_glucneo_1.2.1.13_GAPDHSYNEC__RXN_NAD (-0.87, 0.87) -0.4
NADPH_recycle (-100.0, 1000.0) 0.4
NADH_recycle (-100.0, 1000.0) 0.4
T_flux_pyruvate (0.0, 1000.0) 0.8000000000000002


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### 80 ℃, glucose import reaction lower bound was fixed at 0.1

In [17]:
map_the_RNA_to_model (100,'80 (80) enzyme activities',0.1)

T_flux_H+ (-1000, 1000) 0.4000000000000001
T_trans_H+ (-1000, 1000) 0.4000000000000001
T_flux_beta__D__glucose (-0.1, 0) -0.1
T_ABC__transporter_beta__D__glucose (0, 10.0) 0.1
carb_entner_1.1.1.360_GLUCOSE__1__DEHYDROGENASE__NADP+__RXN (0, 2.5) 0.1
carb_entner_3.1.1.17_GLUCONOLACT__RXN (0, 10.0) 0.10000000000000002
carb_entner_4.2.1.140_GLUCONATE__DEHYDRATASE__RXN (0, 1.7) 0.10000000000000002
carb_entner_2.7.1.178_DEOXYGLUCONOKIN__RXN (0, 10.0) 0.10000000000000002
carb_entner_4.1.2.55_KDPGALDOL__RXN (0, 1.2) 0.10000000000000002
carb_enter_glucneo_5.4.2.1_3PGAREARR__RXN (-0.75, 0.75) 0.1
carb_enter_glucneo_4.2.1.11_2PGADEHYDRAT__RXN (-10.0, 10.0) 0.1
carb_enter_2.7.1.40_PEPDEPHOS__RXN (0, 0.76) 0.1
carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN (-1.1, 1.1) -0.1
carb_glucneo_1.2.1.13_GAPDHSYNEC__RXN_NAD (-0.87, 0.87) -0.1
NADPH_recycle (-100.0, 1000.0) 0.1
NADH_recycle (-100.0, 1000.0) 0.1
T_flux_pyruvate (0.0, 1000.0) 0.20000000000000004


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

## 70 ℃, different a factor

### 70 ℃, a factor was  fixed at 100

In [18]:
map_the_RNA_to_model (100,'70 enzyme activities',1)

T_flux_H+ (-1000, 1000) 1.8300000000000005
T_trans_H+ (-1000, 1000) 1.8300000000000007
T_flux_beta__D__glucose (-1, 0) -0.5100000000000001
T_ABC__transporter_beta__D__glucose (0, 10.0) 0.5100000000000001
carb_entner_1.1.1.360_GLUCOSE__1__DEHYDROGENASE__NADP+__RXN (0, 0.96) 0.51
carb_entner_3.1.1.17_GLUCONOLACT__RXN (0, 10.0) 0.5100000000000001
carb_entner_4.2.1.140_GLUCONATE__DEHYDRATASE__RXN (0, 0.92) 0.5100000000000001
carb_entner_2.7.1.178_DEOXYGLUCONOKIN__RXN (0, 10.0) 0.5100000000000001
carb_entner_4.1.2.55_KDPGALDOL__RXN (0, 0.55) 0.51
carb_enter_glucneo_1.2.1.9_1.2.1.9__RXN (0, 0.21) 0.21
carb_enter_glucneo_5.4.2.1_3PGAREARR__RXN (-0.93, 0.93) 0.5100000000000001
carb_enter_glucneo_4.2.1.11_2PGADEHYDRAT__RXN (-10.0, 10.0) 0.5100000000000001
carb_enter_2.7.1.40_PEPDEPHOS__RXN (0, 0.7) 0.51
carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN (-0.3, 0.3) -0.3
carb_glucneo_1.2.1.13_GAPDHSYNEC__RXN_NAD (-0.43, 0.43) -0.3
NADPH_recycle (-100.0, 1000.0) 0.7200000000000002
NADH_recycle (-100.0, 

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### 70 ℃, a factor was  fixed at 1000

In [19]:
map_the_RNA_to_model (1000,'70 enzyme activities',1)

T_flux_H+ (-1000, 1000) 0.18300000000000008
T_trans_H+ (-1000, 1000) 0.18299999999999997
T_flux_beta__D__glucose (-1, 0) -0.05099999999999998
T_ABC__transporter_beta__D__glucose (0, 1.0) 0.05099999999999999
carb_entner_1.1.1.360_GLUCOSE__1__DEHYDROGENASE__NADP+__RXN (0, 0.096) 0.05099999999999999
carb_entner_3.1.1.17_GLUCONOLACT__RXN (0, 1.0) 0.051
carb_entner_4.2.1.140_GLUCONATE__DEHYDRATASE__RXN (0, 0.092) 0.051
carb_entner_2.7.1.178_DEOXYGLUCONOKIN__RXN (0, 1.0) 0.051
carb_entner_4.1.2.55_KDPGALDOL__RXN (0, 0.055) 0.051000000000000004
carb_enter_glucneo_1.2.1.9_1.2.1.9__RXN (0, 0.021) 0.021
carb_enter_glucneo_5.4.2.1_3PGAREARR__RXN (-0.093, 0.093) 0.051000000000000004
carb_enter_glucneo_4.2.1.11_2PGADEHYDRAT__RXN (-1.0, 1.0) 0.051
carb_enter_2.7.1.40_PEPDEPHOS__RXN (0, 0.07) 0.051
carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN (-0.03, 0.03) -0.03
carb_glucneo_1.2.1.13_GAPDHSYNEC__RXN_NAD (-0.043, 0.043) -0.030000000000000002
NADPH_recycle (-100.0, 1000.0) 0.07199999999999998
NADH_recyc

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### 70 ℃, a factor was  fixed at 10

In [20]:
map_the_RNA_to_model (10,'70 enzyme activities',1)

T_flux_H+ (-1000, 1000) 4.000000000000002
T_trans_H+ (-1000, 1000) 4.000000000000001
T_flux_beta__D__glucose (-1, 0) -1.0
T_ABC__transporter_beta__D__glucose (0, 100.0) 1.0000000000000002
carb_entner_1.1.1.360_GLUCOSE__1__DEHYDROGENASE__NADP+__RXN (0, 9.6) 1.0000000000000002
carb_entner_3.1.1.17_GLUCONOLACT__RXN (0, 100.0) 1.0000000000000002
carb_entner_4.2.1.140_GLUCONATE__DEHYDRATASE__RXN (0, 9.2) 1.0000000000000002
carb_entner_2.7.1.178_DEOXYGLUCONOKIN__RXN (0, 100.0) 1.0000000000000002
carb_entner_4.1.2.55_KDPGALDOL__RXN (0, 5.5) 1.0
carb_enter_glucneo_5.4.2.1_3PGAREARR__RXN (-9.3, 9.3) 1.0000000000000002
carb_enter_glucneo_4.2.1.11_2PGADEHYDRAT__RXN (-100.0, 100.0) 1.0000000000000002
carb_enter_2.7.1.40_PEPDEPHOS__RXN (0, 7.0) 1.0000000000000002
carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN (-3.0, 3.0) -1.0000000000000004
carb_glucneo_1.2.1.13_GAPDHSYNEC__RXN_NAD (-4.3, 4.3) -1.0000000000000002
NADPH_recycle (-100.0, 1000.0) 1.0
NADH_recycle (-100.0, 1000.0) 1.0000000000000002
T_flu

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

## 70 ℃, different glucose import

### 70 ℃, glucose import reaction lower bound was fixed at 1

In [21]:
map_the_RNA_to_model (100,'70 enzyme activities',1)

T_flux_H+ (-1000, 1000) 1.8300000000000005
T_trans_H+ (-1000, 1000) 1.8300000000000007
T_flux_beta__D__glucose (-1, 0) -0.5100000000000001
T_ABC__transporter_beta__D__glucose (0, 10.0) 0.5100000000000001
carb_entner_1.1.1.360_GLUCOSE__1__DEHYDROGENASE__NADP+__RXN (0, 0.96) 0.51
carb_entner_3.1.1.17_GLUCONOLACT__RXN (0, 10.0) 0.5100000000000001
carb_entner_4.2.1.140_GLUCONATE__DEHYDRATASE__RXN (0, 0.92) 0.5100000000000001
carb_entner_2.7.1.178_DEOXYGLUCONOKIN__RXN (0, 10.0) 0.5100000000000001
carb_entner_4.1.2.55_KDPGALDOL__RXN (0, 0.55) 0.51
carb_enter_glucneo_1.2.1.9_1.2.1.9__RXN (0, 0.21) 0.21
carb_enter_glucneo_5.4.2.1_3PGAREARR__RXN (-0.93, 0.93) 0.5100000000000001
carb_enter_glucneo_4.2.1.11_2PGADEHYDRAT__RXN (-10.0, 10.0) 0.5100000000000001
carb_enter_2.7.1.40_PEPDEPHOS__RXN (0, 0.7) 0.51
carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN (-0.3, 0.3) -0.3
carb_glucneo_1.2.1.13_GAPDHSYNEC__RXN_NAD (-0.43, 0.43) -0.3
NADPH_recycle (-100.0, 1000.0) 0.7200000000000002
NADH_recycle (-100.0, 

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### 70 ℃, glucose import reaction lower bound was fixed at 0.4

In [22]:
map_the_RNA_to_model (100,'70 enzyme activities',0.4)

T_flux_H+ (-1000, 1000) 1.5000000000000002
T_trans_H+ (-1000, 1000) 1.5000000000000002
T_flux_beta__D__glucose (-0.4, 0) -0.4
T_ABC__transporter_beta__D__glucose (0, 10.0) 0.4
carb_entner_1.1.1.360_GLUCOSE__1__DEHYDROGENASE__NADP+__RXN (0, 0.96) 0.4
carb_entner_3.1.1.17_GLUCONOLACT__RXN (0, 10.0) 0.4000000000000001
carb_entner_4.2.1.140_GLUCONATE__DEHYDRATASE__RXN (0, 0.92) 0.4000000000000001
carb_entner_2.7.1.178_DEOXYGLUCONOKIN__RXN (0, 10.0) 0.4000000000000001
carb_entner_4.1.2.55_KDPGALDOL__RXN (0, 0.55) 0.4000000000000001
carb_enter_glucneo_1.2.1.9_1.2.1.9__RXN (0, 0.21) 0.10000000000000006
carb_enter_glucneo_5.4.2.1_3PGAREARR__RXN (-0.93, 0.93) 0.4
carb_enter_glucneo_4.2.1.11_2PGADEHYDRAT__RXN (-10.0, 10.0) 0.4
carb_enter_2.7.1.40_PEPDEPHOS__RXN (0, 0.7) 0.4
carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN (-0.3, 0.3) -0.3
carb_glucneo_1.2.1.13_GAPDHSYNEC__RXN_NAD (-0.43, 0.43) -0.3
NADPH_recycle (-100.0, 1000.0) 0.5000000000000001
NADH_recycle (-100.0, 1000.0) 0.3
T_flux_pyruvate (0.

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### 70 ℃, glucose import reaction lower bound was fixed at 0.1

In [23]:
map_the_RNA_to_model (100,'70 enzyme activities',0.1)

T_flux_H+ (-1000, 1000) 0.4000000000000001
T_trans_H+ (-1000, 1000) 0.4000000000000001
T_flux_beta__D__glucose (-0.1, 0) -0.1
T_ABC__transporter_beta__D__glucose (0, 10.0) 0.1
carb_entner_1.1.1.360_GLUCOSE__1__DEHYDROGENASE__NADP+__RXN (0, 0.96) 0.1
carb_entner_3.1.1.17_GLUCONOLACT__RXN (0, 10.0) 0.10000000000000002
carb_entner_4.2.1.140_GLUCONATE__DEHYDRATASE__RXN (0, 0.92) 0.10000000000000002
carb_entner_2.7.1.178_DEOXYGLUCONOKIN__RXN (0, 10.0) 0.10000000000000002
carb_entner_4.1.2.55_KDPGALDOL__RXN (0, 0.55) 0.10000000000000002
carb_enter_glucneo_5.4.2.1_3PGAREARR__RXN (-0.93, 0.93) 0.1
carb_enter_glucneo_4.2.1.11_2PGADEHYDRAT__RXN (-10.0, 10.0) 0.1
carb_enter_2.7.1.40_PEPDEPHOS__RXN (0, 0.7) 0.1
carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN (-0.3, 0.3) -0.1
carb_glucneo_1.2.1.13_GAPDHSYNEC__RXN_NAD (-0.43, 0.43) -0.1
NADPH_recycle (-100.0, 1000.0) 0.1
NADH_recycle (-100.0, 1000.0) 0.1
T_flux_pyruvate (0.0, 1000.0) 0.20000000000000004


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…